In [1]:
import quantstats as qs
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.colors import LinearSegmentedColormap
from quantstats.stats import *
from quantstats._plotting.wrappers import distribution
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import df2img
from datetime import date
from datetime import datetime

# my imports
import sys
sys.path.insert(1, '../scripts')
import utils as utl
import html_evaluation_portfolio as hep
import volatility_estimator as ve
import mean_rev_utl as mru

In [2]:
#parameters
short = 20
long = 120

USequity_list = ['SPY', 'DIA', 'QQQ', 'MDY', 'IJR', 'IWC', 'VUG',
                'VGT', 'ITOT', 'XLV', 'VNQ', 'IWD', 'XLE', 'XLF']
GLOBequity_list = ['ACWI', 'ADRE', 'SPDW', 'VEA', 'IEFA', 'DGT', 'IOO',
                'DIM', 'EFV', 'VWO', 'FM', 'SPEU', 'ASEA', 'SCZ', 'GWX']
bond_list = ['TLT', 'BND', 'TIP', 'PHB', 'BWX', 'VCSH', 'GBF', 
            'BSV', 'PLW', 'IGIB', 'IGSB', 'WIP', 'PWZ', 'CWB', 'SPTI']
commodities_list = ['DBB', 'GLD', 'SLV', 'PPLT', 'DBA', 'DBO', 'UNG', 
                   'CORN', 'SOYB', 'DBE', 'USO', 'UGA', 'GRU', 'GSG', 'FUE']
currencies_list = ['UUP', 'CYB', 'FXB', 'FXE', 'FXY', 'FXA', 'ULE', 'FXC',
              'FXF', 'DVB', 'UND','CEW', 'USDU', 'EUO', 'YCL']

list_securities_kama = USequity_list + GLOBequity_list + commodities_list + currencies_list
list_securities_SMA = bond_list
debug_xl = False

start_date = '2006-01-03'
end_date = "2020-11-26"

In [3]:
US_equity = pd.read_csv('../data/key_market_USequties.csv', index_col = 0, header = [0,1])
global_equity = pd.read_csv('../data/key_market_GLOBequties.csv', index_col = 0, header = [0,1])
bonds = pd.read_csv('../data/key_market_bonds.csv', index_col = 0, header = [0,1])
currencies = pd.read_csv('../data/key_market_currencies.csv', index_col = 0, header = [0,1])
commodities = pd.read_csv('../data/key_market_commodities.csv', index_col = 0, header = [0,1])
df_tot = pd.concat([US_equity, global_equity, bonds, currencies, commodities], axis = 1)
df_close = df_tot.iloc[:, df_tot.columns.get_level_values(1)=='Close']
df_close.columns = df_close.columns.droplevel(1)
data = df_close[start_date:end_date]
data_kama = data[list_securities_kama] 
data_SMA = data[list_securities_SMA] 

In [4]:
#compute unique names
title = 'KAMA & SMA '
unique_name = 'KAMA'
for i in list_securities_kama:
        unique_name += '_' + i
unique_name += '_SMA_'
for i in list_securities_SMA:
        unique_name += '_' + i
#title += 'Global equities report'
unique_name = unique_name + '_short_' + str(short) + '_long_'+ str(long)
#unique_name = 'all_etfs'

### Compute SMA strategy + inverse volatility weight

In [21]:
window = 30 #days
estimator = 'satchell'

if estimator == 'satchell':
    data_ohlc = mru.filter_dataframe_holc(df_tot, list_securities_SMA)
    data_ohlc = data_ohlc[start_date: end_date]
    df_satchell_volat = ve.compute_satchell_volatility(data_ohlc, list_securities_SMA, window)
    df_satchell_volat = ve.index_df(df_satchell_volat, data_SMA, window)
    df_satchell_inv_volat = df_satchell_volat.applymap(lambda x: 1/x)
    df_weighted = hep.df_wheighted(df_satchell_inv_volat)

ZeroDivisionError: float division by zero

In [5]:
signal_df_SMA = mru.create_mean_rev_signal(data_SMA, long, short)[0]
data_SMA_r = data_SMA.div(data_SMA.shift(1))-1 #returns of securities
#data_earnings_SMA = hep.df_earnings((signal_df_SMA*df_weighted), data_SMA_r)
data_earnings_SMA = hep.df_earnings(signal_df_SMA, data_SMA_r)
data_earnings_SMA.index = pd.to_datetime(data_earnings_SMA.index)

In [6]:
data_earnings_SMA

,TLT,BND,TIP,PHB,BWX,VCSH,GBF,BSV,PLW,IGIB,IGSB,WIP,PWZ,CWB,SPTI,Tot
Date,,,,,,,,,,,,,,,,
2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19,-0.005149,-0.002043,0.001037,0.004717,0.000657,0.001085,-0.001675,0.000242,-0.002847,0.003265,0.001091,0.001434,0.003234,0.016044,-0.000606,0.020486
2020-11-20,-0.008870,-0.000906,0.002230,-0.002608,0.001969,-0.000361,-0.002549,0.000242,-0.004387,-0.001139,-0.000545,0.003759,0.001433,0.005573,-0.000605,-0.006766
2020-11-23,0.004582,0.000792,-0.000795,0.004303,-0.003274,0.000000,0.000239,0.000000,0.002648,0.000652,0.000182,-0.004637,0.000932,0.013328,0.000908,0.019858


### Compute KAMA strategy

In [8]:
kama_signal_df = mru.create_kama_signal_df(data_kama)
#kama_signal_df.index = pd.to_datetime(kama_signal_df.index)
kama_signal_df['n_sec_active'] = kama_signal_df.apply(mru.number_nan, axis = 1)
kama_signal_df = kama_signal_df.apply(lambda row: row/row['n_sec_active'], axis = 1)
kama_signal_df.drop(columns =['n_sec_active'], inplace = True)

In [9]:
data_kama_r = data_kama.div(data_kama.shift(1))-1 #returns of securities
data_earnings_kama = hep.df_earnings(kama_signal_df, data_kama_r)
data_earnings_kama.index = pd.to_datetime(data_earnings_kama.index)

In [10]:
data_earnings_kama

,SPY,DIA,QQQ,MDY,IJR,IWC,VUG,VGT,ITOT,XLV,...,ULE,FXC,FXF,DVB,UND,CEW,USDU,EUO,YCL,Tot
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2006-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19,-0.000191,-0.000088,-0.000356,-0.000406,-0.000267,-0.000194,-0.000356,-0.000498,-0.000252,-0.000033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005013
2020-11-20,0.000311,0.000331,0.000311,0.000031,0.000049,-0.000266,0.000336,0.000365,0.000239,-0.000104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002348
2020-11-23,-0.000272,-0.000522,-0.000002,-0.000706,-0.001057,-0.000984,-0.000065,-0.000029,-0.000369,-0.000117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011120


In [16]:
data_SMA_r.index, signal_df_SMA.index

(DatetimeIndex(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06',
                '2006-01-09', '2006-01-10', '2006-01-11', '2006-01-12',
                '2006-01-13', '2006-01-17',
                ...
                '2020-11-12', '2020-11-13', '2020-11-16', '2020-11-17',
                '2020-11-18', '2020-11-19', '2020-11-20', '2020-11-23',
                '2020-11-24', '2020-11-25'],
               dtype='datetime64[ns]', name='Date', length=3752, freq=None),
 Index(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06', '2006-01-09',
        '2006-01-10', '2006-01-11', '2006-01-12', '2006-01-13', '2006-01-17',
        ...
        '2020-11-12', '2020-11-13', '2020-11-16', '2020-11-17', '2020-11-18',
        '2020-11-19', '2020-11-20', '2020-11-23', '2020-11-24', '2020-11-25'],
       dtype='object', name='Date', length=3752))